# テクニカル指標による売買シグナルのチェック（株価日足データ）

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import pandas_ta as ta
import matplotlib.dates as mdates
from datetime import date, datetime, timedelta
import numpy as np
plt.style.use('fivethirtyeight')
import pickle
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [2]:
# 銘柄コード辞書を読み込む
with open('JPX_code_dict.pickle', mode='rb') as f:
    jpx = pickle.load(f)

In [3]:
# チェック対象銘柄リストを読み込む
df01 = pd.read_excel('personal_invest.xlsx')
df02 = pd.read_excel('popular_invest.xlsx')
df1 = pd.concat([df01, df02], axis=0)

In [4]:
# 米国チェック対象銘柄リストを読み込む
df04 = pd.read_excel('personal_invest_us.xlsx')
df2 = df04[df04['flag'] == 1]
df2.tail()

,category,name,code,flag,source
35,電力(カナダ),アルゴンキンパワーアンドユーティリティーズ,AQN,1.0,連続増配
36,タバコ,アルトリアグループ,MO,1.0,連続増配
37,エネルギー,ワンオーク,OKE,1.0,連続増配
38,タバコ,ユニバーサル,UVV,1.0,連続増配
39,通信,ベライゾンコミュニケーションズ,VZ,1.0,連続増配


In [5]:
# 米国銘柄リストより仮辞書を作成して日本の辞書に結合する
df2.loc[:,'description'] = df2.copy().loc[:,'name'] + '：' + df2.copy().loc[:,'category']
df3 = df2.set_index('code')
usadd = df3.to_dict()['description']
jpx.update(usadd)

C:\Users\user\AppData\Local\Temp\ipykernel_13960\510649325.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.loc[:,'description'] = df2.copy().loc[:,'name'] + '：' + df2.copy().loc[:,'category']


In [6]:
# コード番号からTickerリストを作成
df1['ticker'] = df1['code'].astype(str)+'.T'
list_ticker = df1['ticker'].tolist() + df2['code'].tolist()
print(list_ticker)

['9531.T', '9432.T', '7419.T', '8267.T', '2730.T', '2340.T', '9726.T', '8411.T', '4182.T', '8593.T', '2317.T', '7466.T', '8089.T', '9006.T', '6742.T', '2503.T', '7186.T', '7201.T', '8131.T', '5411.T', '4004.T', '2432.T', '6632.T', '2002.T', '5991.T', '1963.T', '7011.T', '9928.T', '7013.T', '2201.T', '1942.T', '6617.T', '9507.T', '9502.T', '9503.T', '9508.T', '9506.T', '9504.T', '9509.T', '9501.T', '8306.T', '9509.T', '6740.T', '4005.T', '5020.T', '9107.T', '5032.T', '8604.T', '4755.T', '8308.T', '4503.T', '6178.T', '7267.T', '3436.T', '4689.T', '4188.T', '7211.T', '4751.T', '4385.T', '2181.T', '6752.T', '7261.T', '7182.T', '3402.T', '6503.T', '7003.T', '6902.T', '8410.T', '3382.T', '8802.T', '7269.T', '6971.T', '7733.T', '4543.T', '8601.T', '7272.T', '5019.T', '9613.T', '6702.T', 'CHD', 'COST', 'DHR', 'HD', 'LMT', 'MKC', 'NKE', 'UNH', 'V', 'DUK', 'NEE', 'SO', 'JNJ', 'PG', 'ROL', 'UNP', 'ENTG', 'ETN', 'AAPL', 'AMZN', 'CTAS', 'GOOGL', 'ISRG', 'META', 'MSFT', 'NFLX', 'NVDA', 'ORCL', 'TSLA

## 最新の指標値を返す関数

In [7]:
def func_osc(ticker,start,end):
    res = pd.DataFrame()
    # 新しくインストールしたyfinanceの場合
    df = yf.download(ticker, start, end)
    # 古いyfinanceの場合
    #yf.pdr_override()
    #df = pdr.get_data_yahoo(ticker, start, end)
    # Calculate SMA
    df['SMA_5'] = df.ta.sma(close='Close', length=5)
    df['SMA_25'] = df.ta.sma(close='Close', length=25)
    # Calculate MACD
    df.ta.macd(close='Close', fast=12, slow=26, signal=9, append=True)
    # Calculate Bollinger Bands
    df.ta.bbands(close='Close', length=20, std=2, append=True)
    # Calculate RSI
    df['RSI_14'] = ta.rsi(df['Close'], length=14)
    df['ticker'] = ticker
    return df

## 指標値の計算

In [8]:
df = pd.DataFrame()
end = datetime.today()
start = end - timedelta(days=1350)
for ticker in list_ticker:
    df1 = func_osc(ticker,start,end)
    # 一時点前の値が必要な場合はここで記述する
    df1['close_p'] = df1['Close'].shift(1)
    df1['SMA_5_p'] = df1['SMA_5'].shift(1)
    df1['SMA_25_p'] = df1['SMA_25'].shift(1)
    df1['MACD_12_26_9_p'] = df1['MACD_12_26_9'].shift(1)
    df1['MACDs_12_26_9_p'] = df1['MACDs_12_26_9'].shift(1)
    df1['BBU_20_2.0_p'] = df1['BBU_20_2.0'].shift(1)
    df1['BBL_20_2.0_p'] = df1['BBL_20_2.0'].shift(1)
    df1['RSI_14_p'] = df1['RSI_14'].shift(1)
    df = pd.concat([df,df1.tail(1)],axis=0)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [9]:
df.head()

,Open,High,Low,Close,Adj Close,Volume,SMA_5,SMA_25,MACD_12_26_9,MACDh_12_26_9,...,RSI_14,ticker,close_p,SMA_5_p,SMA_25_p,MACD_12_26_9_p,MACDs_12_26_9_p,BBU_20_2.0_p,BBL_20_2.0_p,RSI_14_p
Date,,,,,,,,,,,,,,,,,,,,,
2024-08-16,3534.000000,3572.000000,3511.000000,3557.0,3557.0,1665200,3459.800000,3341.760000,17.621891,40.448948,...,63.185983,9531.T,3495.000000,3423.800000,3335.920000,2.021615,-32.939294,3549.664616,3083.035384,59.873108
2024-08-16,151.100006,151.800003,149.800003,151.0,151.0,167453900,148.479999,154.599998,-2.140474,-0.445284,...,47.180537,9432.T,149.300003,147.359998,154.891998,-2.388518,-1.583869,165.861286,142.028710,43.804789
2024-08-16,1622.000000,1645.000000,1607.000000,1645.0,1645.0,331800,1589.800000,1601.520000,-5.780966,3.830383,...,57.197601,7419.T,1588.000000,1570.200000,1598.400000,-11.791852,-10.568945,1680.578306,1515.921694,49.133189
2024-08-16,3453.000000,3489.000000,3433.000000,3481.0,3481.0,1930400,3481.600000,3372.120000,26.183329,14.474642,...,56.128052,8267.T,3430.000000,3479.000000,3366.680000,23.513069,8.090027,3533.129623,3222.870377,52.381734
2024-08-16,1774.000000,1781.000000,1762.000000,1781.0,1781.0,318500,1776.000000,1739.760000,28.609789,-1.826885,...,58.997087,2730.T,1763.000000,1775.400000,1734.960000,28.932604,30.893395,1808.172731,1691.127269,56.369074


## シグナルの判定

In [10]:
# 移動平均のゴールデンクロス
df['MA_buy'] = np.where((df['SMA_5_p']<=df['SMA_25_p']) & (df['SMA_5']>df['SMA_25']), 100, 0)
# 移動平均のデッドクロス
df['MA_sell'] = np.where((df['SMA_5_p']>=df['SMA_25_p']) & (df['SMA_5']<df['SMA_25']), 100, 0)
# MACDのゴールデンクロス
df['MACD_buy'] = np.where((df['MACD_12_26_9_p']<=df['MACDs_12_26_9_p']) & (df['MACD_12_26_9']>df['MACDs_12_26_9']), 100, 0)
# MACDのデッドデンクロス
df['MACD_sell'] = np.where((df['MACD_12_26_9_p']>=df['MACDs_12_26_9_p']) & (df['MACD_12_26_9']<df['MACDs_12_26_9']), 100, 0)
# 下方ボリンジャーバンド割れ
df['Bollinger_buy'] = np.where((df['close_p']>=df['BBL_20_2.0_p']) & (df['Close']<df['BBL_20_2.0']), 100, 0)
# 上方ボリンジャーバンド超え
df['Bollinger_sell'] = np.where((df['close_p']<=df['BBU_20_2.0_p']) & (df['Close']>df['BBU_20_2.0']), 100, 0)
# 相対力指数の下限値超え
df['RSI_buy'] = np.where((df['RSI_14_p']<=25) & (df['RSI_14']>25), 100, 0)
# 相対力指数の上限値超え
df['RSI_sell'] = np.where((df['RSI_14_p']<=75) & (df['RSI_14']>75), 100, 0)

## コードから銘柄名を返す関数

In [11]:
def ticker_dict(df):
    return jpx.get(df)

## 銘柄名の付与

In [12]:
df_signal = df[['ticker','Close','MA_buy','MA_sell','MACD_buy','MACD_sell','Bollinger_buy','Bollinger_sell','RSI_buy','RSI_sell']].reset_index()
df_signal['ticker2'] = df_signal['ticker'].mask(df_signal['ticker'].str.endswith('.T'),df_signal['ticker'].str[:-2])
df_signal['cname'] = df_signal['ticker2'].map(ticker_dict)
df_signal

,Date,ticker,Close,MA_buy,MA_sell,MACD_buy,MACD_sell,Bollinger_buy,Bollinger_sell,RSI_buy,RSI_sell,ticker2,cname
0,2024-08-16,9531.T,3557.000000,0,0,0,0,0,0,0,0,9531,東京瓦斯
1,2024-08-16,9432.T,151.000000,0,0,0,0,0,0,0,0,9432,日本電信電話
2,2024-08-16,7419.T,1645.000000,0,0,100,0,0,0,0,0,7419,ノジマ
3,2024-08-16,8267.T,3481.000000,0,0,0,0,0,0,0,0,8267,イオン
4,2024-08-16,2730.T,1781.000000,0,0,0,0,0,0,0,0,2730,エディオン
...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,2024-08-16,AQN,5.350000,0,0,0,0,0,0,0,0,AQN,アルゴンキンパワーアンドユーティリティーズ：電力(カナダ)
114,2024-08-16,MO,51.320000,0,0,0,0,0,0,0,0,MO,アルトリアグループ：タバコ
115,2024-08-16,OKE,87.459999,0,0,0,0,0,0,0,0,OKE,ワンオーク：エネルギー
116,2024-08-16,UVV,53.230000,0,0,100,0,0,0,0,0,UVV,ユニバーサル：タバコ


## シグナルメッセージの付与

In [13]:
df_signal["signal_str"] = np.nan
df_signal["signal_total"] = np.nan
for irow in range(len(df_signal)):
    str_signal = ''
    total_signal = 0
    if df_signal.MA_buy[irow] == 100:
        str_signal += 'MA買い,'
        total_signal += 100
    if df_signal.MA_sell[irow] == 100:
        str_signal += 'MA売り,'
        total_signal += 100
    if df_signal.MACD_buy[irow] == 100:
        str_signal += 'MACD買い,'
        total_signal += 100
    if df_signal.MACD_sell[irow] == 100:
        str_signal += 'MACD売り,'
        total_signal += 100
    if df_signal.Bollinger_buy[irow] == 100:
        str_signal += 'ボリ買い,'
        total_signal += 100
    if df_signal.Bollinger_sell[irow] == 100:
        str_signal += 'ボリ売り,'
        total_signal += 100
    if df_signal.RSI_buy[irow] == 100:
        str_signal += 'RSI買い,'
        total_signal += 100
    if df_signal.RSI_sell[irow] == 100:
        str_signal += 'RSI売り,'
        total_signal += 100
    df_signal.loc[irow,"signal_str"] = str_signal
    df_signal.loc[irow,"signal_total"] = total_signal

## シグナル一覧表の表示

In [14]:
df_tmp = df_signal[['Date', 'ticker', 'cname', 'signal_str', 'signal_total']]
df_table = df_tmp[df_tmp['signal_total'] > 0]
df_table

,Date,ticker,cname,signal_str,signal_total
2,2024-08-16,7419.T,ノジマ,"MACD買い,",100.0
7,2024-08-16,8411.T,みずほフィナンシャルグループ,"MACD買い,",100.0
10,2024-08-16,2317.T,システナ,"MACD買い,ボリ売り,",200.0
15,2024-08-16,2503.T,キリンホールディングス,"MACD買い,",100.0
16,2024-08-16,7186.T,コンコルディア・フィナンシャルグループ,"MACD買い,",100.0
17,2024-08-16,7201.T,日産自動車,"MACD買い,",100.0
21,2024-08-16,2432.T,ディー・エヌ・エー,"ボリ売り,",100.0
25,2024-08-16,1963.T,日揮ホールディングス,"MA買い,",100.0
27,2024-08-16,9928.T,ミロク情報サービス,"MACD買い,",100.0
28,2024-08-16,7013.T,ＩＨＩ,"MA買い,",100.0


【注意】
RSIによる判定は、株価が明確なトレンドに乗っているいる場合は無効、またRSIが一旦底をついてから上がったのではない場合も無効。
MACDは弱いトレンドや横ばいトレンドではあまり有効ではない。ボリンジャーバンドによる逆張りは株価が明確なトレンドに乗っているいる場合は無効。ボリンジャー適用にあたってはシグナルとともにローソク足の影が長い(支配的な動きがない)かを確かめる。

In [15]:
df_table.to_csv('SignalToday_d.csv')